In [1]:
import pandas as pd
import numpy as np
import gurobipy as gp
import matplotlib.pyplot as plt

In [2]:
timeLimit = 600

## Problem Overview
One of the most common problems in predictive analytics is variable selection for regression. Direct variable selection using optimization has long been dismissed by the statistics/analytics community because of computational difficulties. This computational issue was part of the motivation for the development of LASSO and ridge regression. However, in the recent past there have been tremendous advancements in optimization software, specifically the ability to solve mixed integer quadratic programs (MIQP). This project will pose the variable selection problem for regression as an MIQP which you will solve using gurobi. You will compare the results you find to LASSO to see if the additional ‘shrinkage’ component of LASSO really is more beneficial than finding the ‘best’ set of variables to include in your regression.

## Direct Variable Selection – MIQP Problem
Given a dataset of $m$ independent variables, $X$, and a dependent variable, $y$, the standard ordinary least squares problem is formulated as

$$\min_\beta \sum^{n}_{i=1}(\beta_0+\beta_1 x_{i1}+...+\beta_m x_{im}-y_i)^2$$

In order to incorporate variable selection into this problem we can include some binary variables, $z_j$, that force the corresponding values of $\beta$ to be zero if $z_j$ is zero, using the big-M
method that we discussed in class, and used in the previous project. If we only want to include at most $k$ variables from $X$, then we can pose this as

$$\min_{\beta,z} \sum^{n}_{i=1}(\beta_0+\beta_1 x_{i1}+...+\beta_m x_{im}-y_i)^2$$
$$s.t. -Mz_j\leq\beta_j\leq Mz_j, for j=1,2,...,m$$

$$\sum_{j=1}^m z_j\leq k$$
$$z_j=binary$$

Note that we don’t ever forbid the model from having an intercept term, $\beta_0$, and that $m$ and $M$ are different things here. Here, $k$ can be viewed as a hyperparameter to be chosen using cross validation.

In order to pose this in the standard framework of a quadratic programming objective let’s see how we can rewrite this objective using linear algebra. Let $\beta$ be an $(m+1) \times 1$ column vector that contains $\beta_0,...\beta_m$, let X be the $n \times (m+1)$ matrix that has its **first column made up entirely of 1s**, and columns 2 to (m+1) have the data from the m independent variables, and let y be the $n\times 1$ column vector that has the dependent variable data. You can use np.array to convert the pandas dataframe to a matrix and then add a column of all 1s to the matrix. Then we can create an n x 1 vector whose entries are the n values inside the parentheses from the problem statement by doing the following matrix calculation: $(X\beta − y)$. Then if we want to take the sum of squared entries of this vector, we can multiply $(X\beta − y)^T ∗ (X\beta − y)$. Using a few tricks from linear algebra we can pose the optimization problem’s objective function as

$$\min_{\beta,z}\beta^T(X^TX)\beta + (-2y^TX)\beta$$

The only issue left to be resolved is that the vector of decision variables needs to be of size $(2m+1)\times 1$; made up of the $(m+1)$ values of $\beta$ and the $m$ values of $z$, but the objective written above only includes the $m+1$ values of $\beta$. To fix this we can assign the $Q$ matrix to be a $(2m+1) \times (2m+1)$ matrix where the upper left corner of the matrix is equal to $X^TX$, and all other values are zero. We also need the linear term of the objective to be a $(2m+1)\times 1$ vector where the first $(m+1)$ components are $-2y^TX$, and the rest are zeros. Now if you create the constraint matrix and right-hand-side vector from the constraints given above, you have all you need to solve the problem using gurobi.

## Indirect Variable Selection – LASSO
The LASSO version of regression is posed as
$$\min_\beta \sum^{n}_{i=1}(\beta_0+\beta_1 x_{i1}+...+\beta_m x_{im}-y_i)^2+\lambda \sum^m_{j=1}|\beta_j|$$
where $\lambda$ is a hyperparameter to be chosen using cross-validation. It turns out that if $\lambda$ is large enough, several values of $\beta$ will be forced to be equal to zero. This model also has the benefit of ‘shrinking’ the $\beta$s closer to zero, which achieves variance reduction (prevents overfitting). Note again that $\beta_0$ is not included in the $\lambda$ sum. You should never penalize a model for having an intercept term. The standard package in Python to solve the LASSO problem is scikit learn. In this project you will need to use scikit learn to solve the LASSO problem.

## 1
One data set is a training data set, and one is a test data set. You will follow the data science pipeline carefully here. You will first do 10-fold cross validation on the training set to pick $k$ or $\lambda$. Then using the optimal values of $k$ or $\lambda$ you will fit your $\beta$s using the entire training set. Then with those $\beta$s you will make a prediction of the y values on the test set, and compare your prediction of y, to the true value of y in the test set.

In [3]:
train = pd.read_csv('training_data.csv')
test = pd.read_csv('test_data.csv')

In [4]:
print(train.shape, test.shape)

(250, 51) (50, 51)


In [5]:
y = np.array(train['y'])
X = np.array(train.iloc[:, 1:])

## 2
In order to do cross validation on the MIQP model you will have to write your own cross validation code. Randomly shuffle your data and split it into 10 folds (‘np.random.choice()’). There are 50 X variables, and you will need to try k = 5, 10, 15, ..., 50 in your cross validation. This means to do 10-fold cross validation with all possible values of k, you will have to solve an MIQP model 100 times! Pick the value of k that corresponds to the smallest cross validation error: for a given value of k, sum each validation set’s sum of squared errors using the $\beta$s found using the other 9 folds’ data to solve the MIQP. When k is 5 or 50, gurobi should solve the problem pretty quickly, but when k is 25 it will probably take a long time. Therefore, you should set a time limit for gurobi to solve each problem. Don’t let the entire process run for any longer than 12 hours. 

> a. Set lb value of your model to be -M for the appropriate decision variables.

> b. Choose M to be large enough so that no value of $\beta$ is equal to M or -M. If you solve the problem and one of your $\beta$s is M or -M then you should double M and resolve the problem. Repeat until no $\beta$ is equal to M or -M.

In [6]:
from sklearn.model_selection import KFold
import os

In [7]:
if os.path.exists('k_sse.csv'):
    k_sse = pd.read_csv('k_sse.csv') 
    print(k_sse)
else:
    ks = [x for x in range(5,51,5)]
    kf = KFold(n_splits=10)  # if we need other number of folds, change it here
    error_dict = {}

    for k in ks:
        error = 0
        for train_index, test_index in kf.split(X):

            X_train, X_validate = X[train_index], X[test_index]
            y_train, y_validate = y[train_index], y[test_index]

            # big M
            M = 100

            # m and n
            n, m = X_train.shape

            # add column of 1's in X
            X_Q = np.ones((n,m+1))
            X_Q[:,1:] = X_train

            # quadratic objective
            Q = np.zeros((2*m+1, 2*m+1))
            Q[:m+1, :m+1] = X_Q.T @ X_Q

            # linear objective
            C = np.zeros(2*m+1)
            C[:m+1] = (-2)*y_train.T @ X_Q

            # decision variables
            # beta0, beta1, ..., betam, z1, ..., zm
            vtypes = np.array(['C']*(m+1) + ['B']*m)
            lb = np.array([-np.inf] + [-M]*m + [0]*m)
            
            # constraint matrix
            A = np.zeros((2*m+1, 2*m+1))
            A[0, m+1:] = [1]*m                    # z1 + ... + zm ≤ k
            np.fill_diagonal(A[1:m+1, 1:m+1], 1)  # coefficients for beta_j
            np.fill_diagonal(A[1:m+1, m+1:], M)   # beta_j + Mz_j ≥ 0
            np.fill_diagonal(A[m+1:, 1:m+1], 1)   # coefficients for beta_j
            np.fill_diagonal(A[m+1:, m+1:], -M)   # beta_j - Mz_j ≤ 0

            b = np.zeros((2*m+1,1))
            b[0] = k

            sense = np.array(['<'] + ['>']*m + ['<']*m)

            miqpMod = gp.Model()
            miqpMod_x = miqpMod.addMVar(2*m+1, vtype=vtypes, lb=lb)
            miqpMod_con = miqpMod.addMConstr(A, miqpMod_x, sense, b)
            miqpMod.setMObjective(Q,C,0,sense=gp.GRB.MINIMIZE)
            miqpMod.Params.OutputFlag = 0 
            miqpMod.Params.TimeLimit = timeLimit
            miqpMod.optimize()

            betas = np.array(miqpMod.x[:m+1])

            n_test, m_test = X_validate.shape

            X_Q2 = np.ones((n_test,m_test+1))
            X_Q2[:,1:] = X_validate

            error += (X_Q2 @ betas - y_validate).T @ (X_Q2 @ betas - y_validate)

        error_dict[k] = error
        print(k, ':', error)
    best = min(error_dict, key=error_dict.get)
    print('Best k: {}, SSE: {:.4f}'.format(best, error_dict[best]))
    
    keys = [x[0] for x in error_dict.items()]
    values = [x[1] for x in error_dict.items()]
    k_sse = pd.DataFrame.from_dict({'k':keys, 'sse':values})
    k_sse.to_csv('k_sse.csv', index=False)

    k         sse
0   5  917.479061
1  10  724.787631
2  15  764.049938
3  20  799.003940
4  25  770.482828
5  30  830.082402
6  35  832.484731
7  40  847.343751
8  45  843.258526
9  50  847.184545


## 3
Once you find the k with the smallest cross validation error, fit the MIQP model on the entire training set using that value of k. Use the $\beta$s you find in this MIQP to make a prediction of the y values in the test set.

In [8]:
best_k = int(k_sse['k'][k_sse['sse'] == k_sse['sse'].min()])
best_k

10

In [9]:
X_train, y_train = train.iloc[:, 1:], train['y']

n, m = X_train.shape
M = 100

In [10]:
# add column of 1's in X
X_Q = np.ones((n,m+1))
X_Q[:,1:] = X_train

# quadratic objective
Q = np.zeros((2*m+1, 2*m+1))
Q[:m+1, :m+1] = X_Q.T @ X_Q

# linear objective
C = np.zeros(2*m+1)
C[:m+1] = (-2)*y_train.T @ X_Q

# decision variables
# beta0, beta1, ..., betam, z1, ..., zm
vtypes = np.array(['C']*(m+1) + ['B']*m)
lb = np.array([-np.inf] + [-M]*m + [0]*m)

# constraint matrix
A = np.zeros((2*m+1, 2*m+1))
A[0, m+1:] = [1]*m 
np.fill_diagonal(A[1:m+1, 1:m+1], 1)
np.fill_diagonal(A[1:m+1, m+1:], M)
np.fill_diagonal(A[m+1:, 1:m+1], 1) 
np.fill_diagonal(A[m+1:, m+1:], -M) 

b = np.zeros((2*m+1,1))
b[0] = best_k
sense = np.array(['<'] + ['>']*m + ['<']*m)

In [11]:
miqpMod = gp.Model()
miqpMod_x = miqpMod.addMVar(2*m+1, vtype=vtypes, lb=lb)
miqpMod_con = miqpMod.addMConstr(A, miqpMod_x, sense, b)
miqpMod.setMObjective(Q,C,0,sense=gp.GRB.MINIMIZE)
miqpMod.Params.OutputFlag = 0 
miqpMod.Params.TimeLimit = timeLimit
miqpMod.optimize()

beta = np.array(miqpMod.x[:m+1])

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17


In [12]:
from sklearn.metrics import mean_squared_error

In [13]:
X_test, y_test = test.iloc[:, 1:], test['y']
n_test, m_test = X_test.shape

X_Q = np.ones((n_test,m_test+1))
X_Q[:,1:] = X_test

sse = (X_Q @ beta - y_test).T @ (X_Q @ beta - y_test)
mse = mean_squared_error(y_test, X_Q @ beta)

print('k = {}, SSE = {:.5f}, MSE = {:.5f}'.format(best_k, sse, mse))

k = 10, SSE = 116.82720, MSE = 2.33654


## 4
Use scikit learn to do 10-fold cross validation on the training set to pick $\lambda$. Once you find the best value of $\lambda$, fit a LASSO model to the entire training set using that value of $\lambda$. With the $\beta$s you find in that LASSO model make a prediction of the y values in the test set.

In [14]:
from sklearn.linear_model import LassoCV

In [55]:
lasso = LassoCV(cv=10, normalize=True).fit(X_train, y_train)

alpha, beta = lasso.alpha_, lasso.coef_
print('Best lambda: {}\nNumber of variables: {}'.format(alpha, (beta != 0).sum()))

Best lambda: 0.0057453437864455085
Number of variables: 18
[-0.         -0.          0.          0.         -0.          0.
 -0.         -0.         -2.11561506  0.         -0.06043079 -0.
 -0.         -0.         -0.41674549 -0.18155256  0.          0.
 -0.          0.          0.         -0.19710223 -1.3655275   0.73510021
 -0.         -1.30018578  0.          0.          0.06390289  0.
 -0.          0.         -0.10737966  0.25392747  0.02138366  0.
  0.          0.         -0.21159473  0.         -0.          0.
  0.          0.01152326  1.53171531 -0.01408773  0.6504778  -0.09757869
  0.          0.        ]


/Users/louie/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(


In [43]:
y_pred = lasso.predict(X_test)

mse_lasso = mean_squared_error(y_test, y_pred)
print('Lambda = {:.5f}, MSE = {:.5f}'.format(alpha, mse_lasso))

Lambda = 0.00575, MSE = 2.35971
